In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6
%load_ext autoreload
%autoreload 2
import sample_data

np.random.seed(0)
N_train = 2 ** 10; N_test = 0
N = N_train + N_test
train = np.arange(N_train); test = np.arange(N_test) + N_train
n_min = 16; n_max = 16

X, w = sample_data.periodic(N, n_min, n_max, t_max=2*np.pi, even=True,
                            A=1., sigma=1e-0, w_min=0.1, w_max=1.)
M = X[:, :, 1:2]
print(M.shape)

In [ ]:
%%time
Y = np.fft.rfft(M)[:, :, :]
print(M.shape, Y.shape)

In [ ]:
fig, ax = plt.subplots(2, 3, sharey=False)
for i in range(3):
    ax[0, i].plot(X[i][:, 0], M[i, :, 0], 'o')
    ax[0, i].set(xlabel="t", ylabel="y")
    ax[1, i].plot(X[i][:, 0], Y[i, :], 'o')
    ax[1, i].set(xlabel="t", ylabel="y")

In [ ]:
from keras.callbacks import Callback
from IPython.core.display import clear_output

class ClearOutput(Callback):
    def on_epoch_end(self, epoch, logs={}):
        clear_output(wait=True)

In [ ]:
import os
import tensorflow as tf
from keras import backend as K
from keras.layers import (Input, Dense, TimeDistributed, Activation, LSTM,
                          Dropout, merge, Reshape, Flatten, RepeatVector,
                          Convolution1D, MaxPooling1D)
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, activity_l1
from keras import metrics
from keras.callbacks import ProgbarLogger, TensorBoard
from IPython.display import clear_output, display

sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=1.0)))
K.set_session(sess)

#lstm_size = 128
#num_layers = 4
#lr = 4e-3
#drop_frac = 0.2
#run = "lstm_{:03d}_{}_{:1.0e}_drop{}".format(lstm_size, num_layers, lr, drop_frac).replace('e-', 'm').replace('.', '')
#print(run)
#model = Sequential()
#model.add(LSTM(lstm_size, input_shape=(n_max, 1), return_sequences=True))
#for i in range(1, num_layers):
#    model.add(Dropout(drop_frac))
#    model.add(LSTM(lstm_size, return_sequences=(i != num_layers - 1)))
#model.add(Dense(Y.shape[1], activation='linear'))
#adam = Adam(lr=lr)
#model.compile(optimizer=adam, loss='mse', metrics=[])

#model = Sequential()
#model.add(Convolution1D(conv_size, length, border_mode='same', input_shape=(n_max, 1)))
#model.add(MaxPooling1D())
#model.add(Flatten())
#model.add(Dense(Y.shape[1], activation='linear'))
#adam = Adam(lr=lr)
#model.compile(optimizer=adam, loss='mse', metrics=[])
#run = "conv_max_{}_{}_{:1.0e}".format(conv_size, length, lr).replace('e-', 'm')
#print(run)

dense_size = n_max
lr = 3e-2
model = Model()
input = Input(shape=(n_max,))
dense_1 = Dense(dense_size, activation='linear', bias=False, input_shape=(n_max,)))
model.add(Dense(dense_size, activation='linear', bias=False))
sgd = SGD(lr=lr)
model.compile(optimizer=sgd, loss='mse', metrics=[metrics.mean_squared_error])
run = "linear_{}_{:1.0e}".format(dense_size, lr).replace('e-', 'm')
print(run)

from keras.utils.visualize_util import model_to_dot
from IPython.display import SVG

model_dot = model_to_dot(model).create(prog='dot', format='svg')
print('{} MB'.format(model.count_params() * np.dtype('float32').itemsize / 1e6))
print(model.summary())
SVG(model_dot)

log_dir = os.path.expanduser('~/Dropbox/Documents/timeflow/keras_logs/fourier/{}'.format(run))
!rm -rf $log_dir
#if os.path.exists(log_dir):
#    raise Exception("Log directory already exists, not overwriting")
history = model.fit(M[train, :, 0], Y[train, :, 0],
                     nb_epoch=20000, batch_size=N_train, validation_split=0.0,
                     callbacks=[ProgbarLogger(), TensorBoard(log_dir=log_dir, write_graph=False),
                                ClearOutput(),
                               ])
model.save_weights(os.path.join(log_dir, 'weights.h5'), overwrite=True)
#                clear_output(wait=True)